In [2]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

# 2.19 Time series / date functionality

In [5]:
dti = pd.to_datetime(
    ['1.1.2018', np.datetime64('2018-01-01'), datetime.datetime(2018,1,1)]
)
dti[1]

Timestamp('2018-01-01 00:00:00')

In [18]:
dti = pd.date_range('2018-01-01', periods=3, freq='4H')
dti

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 04:00:00',
               '2018-01-01 08:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [19]:
dti = dti.tz_localize('Europe/Berlin')


DatetimeIndex(['2018-01-01 00:00:00+01:00', '2018-01-01 04:00:00+01:00',
               '2018-01-01 08:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', freq=None)

In [20]:
dti.tz_convert('US/Central')

DatetimeIndex(['2017-12-31 17:00:00-06:00', '2017-12-31 21:00:00-06:00',
               '2018-01-01 01:00:00-06:00'],
              dtype='datetime64[ns, US/Central]', freq=None)

In [31]:
idx = pd.date_range("2018-01-01", periods=5, freq="H")
ts = pd.Series(range(len(idx)), index=idx)
print(ts)
ts.resample("2H").count()
print(ts.resample("2H").mean())
ts.resample("2H").median()

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: H, dtype: int64
2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64


2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64

In [43]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
saturday = friday + pd.Timedelta("1 day")
saturday.day_name() # 'Saturday'

# Add 1 business day (Friday --> Monday)
monday = friday + pd.offsets.BDay()
monday.day_name()

'Monday'

In [44]:
pd.Series(range(3), index=pd.date_range('2020', freq='D', periods=3))


2020-01-01    0
2020-01-02    1
2020-01-03    2
Freq: D, dtype: int64

In [45]:
df = pd.DataFrame({"type A":[15], "type B": [20], "date": ["2012-03-01"], "station": ["s1"]})

In [46]:
df

,type A,type B,date,station
0,15,20,2012-03-01,s1


In [9]:
friday = pd.Timestamp("2018-01-05")
friday.day_name()
friday + pd.Timedelta('1 Day')
(friday + pd.offsets.Day()).day_name()

'Saturday'

In [16]:
s = pd.Series(pd.date_range('2020', periods=3))
df = s.to_frame()

In [52]:
df = pd.Series(pd.date_range('1/13/2021', periods=6)).to_frame()
df.columns=["dates"]
df['day'] = df.dates.dt.day_name()
df['weekday'] = df.dates.dt.weekday
df

,dates,day,weekday
0,2021-01-13,Wednesday,2
1,2021-01-14,Thursday,3
2,2021-01-15,Friday,4
3,2021-01-16,Saturday,5
4,2021-01-17,Sunday,6
5,2021-01-18,Monday,0


In [61]:
from pandas.tseries.offsets import Week

print(df['dates'] - Week(1, weekday=2))
df['dates'] - pd.offsets.Week(1, weekday=2)

df['pday'] = np.where(df.dates.eq('2021-01-15'),
         df.dates - pd.offsets.Week(2, weekday=2),
        df.dates - pd.offsets.Week(weekday=2))
df

0   2021-01-06
1   2021-01-13
2   2021-01-13
3   2021-01-13
4   2021-01-13
5   2021-01-13
Name: dates, dtype: datetime64[ns]


,dates,day,weekday,pday
0,2021-01-13,Wednesday,2,2021-01-06
1,2021-01-14,Thursday,3,2021-01-13
2,2021-01-15,Friday,4,2021-01-06
3,2021-01-16,Saturday,5,2021-01-13
4,2021-01-17,Sunday,6,2021-01-13
5,2021-01-18,Monday,0,2021-01-13


In [65]:
df=pd.DataFrame({'Date':['2020-01-01', '2020-01-23', '2020-02-22', '2020-03-04'],  'ID':[1001,1002,1002,1003]})
df["Date"] = pd.to_datetime(df["Date"])
df['Wed'] = np.where(df.ID == 1002,
         df.Date - pd.offsets.Week(3, weekday=2),
         df.Date - pd.offsets.Week(weekday=2))
df

,Date,ID,Wed
0,2020-01-01,1001,2019-12-25
1,2020-01-23,1002,2020-01-08
2,2020-02-22,1002,2020-02-05
3,2020-03-04,1003,2020-02-26
